## Understanding your Convolution network with Visualizations

Covolutional Neural networks has revolutionalised image recogniton task. 
The rise of large convolution neural networks started with AlexNet in 2012, which was created by Alex Krizhevsky, 
Ilya Sutskever, and Geoffrey Hinton, and was the winning entry in ImageNet Large-Scale Visual Recognition Challenge that year. Since then there has been no looking back for researchers in this field, and the results in various areas in Computer Vision are a clear proof of that. From the Face Recognition in your phone to driving your cars, the tremendous power of CNNs is being used to solve many real-world problems.

But even after having large scale neural networks its hard to understand how these models are learning.
So here I have tried to visualize the cnn filters as well the activation for every 10 epochs and understand the changes in this visualizations as training progress.

In [6]:
# -------------------------- set gpu using tf ---------------------------
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)




import keras.backend.tensorflow_backend as K
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator, save_img
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
import numpy as np
import Visualizations
from numpy import expand_dims
import matplotlib.pyplot as plt
from skimage.io import imsave
from activation_funciton_visualization import Activation_Visualizations


batch_size = 512
num_classes = 10
epochs = 1000
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
conv_layer_name = ['conv2d_2', 'conv2d_4', 'conv2d_6']
test_image = x_train[3]
test_image = img = expand_dims(test_image, axis=0)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [7]:
# Let us first create a model

def create_model():
    model = Sequential()
    
    '''
    Here I will be using cfar 10 dataset so the input will be 32x32x3
    this cnn model is demo model you can change as you want, I have kept padding
    as same to retain more information. Also I have found adding droput did increased accuracy,
    
    '''
    
    model.add(Conv2D(32, (3, 3), padding='same',input_shape=x_train.shape[1:])) # 32x32x32
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3), padding='same')) # 32x32x32
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(64, (3, 3), padding='same')) #16x16x64
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3), padding='same')) # 16x16x64
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(128, (3, 3), padding='same')) # 8x8x128
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3, 3), padding='same')) # 8x8x128
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))  
    model.add(Dropout(0.5))


    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes)) # here num_class is 10 so the last layer will have 10 output probabilities 
    model.add(Activation('softmax'))

    # initiate RMSprop optimizer
    opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

    # Let's train the model using RMSprop
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model


This is a fucntion to visualise activation maps for a single test image for every 10 epcohs(you can change it and can make it for each epcoh), Here we can see what particular area of the image the filters are being activated for, Its easier to see this behavior of convolution layers in starting layers, because as you go deeper the pattern captured by the convolution kernel become more and more sparse, As you will see the activation maps for end layers is bit difficult to understand. I was able to capture this maps by making intermediate models and running a test image through it

In [3]:

def Activation_visualistation(image , model, layer_name):
    def draw_activations(activations, predict_image):
        ix = 1
        square = int(np.floor(np.sqrt(activations))) # This will be the dimension of the grid
        for _ in range(square):
            for _ in range(square):
                # specify subplot and turn of axis
                ax = plt.subplot(square, square, ix)
                ax.set_xticks([])
                ax.set_yticks([])
                # plot filter channel in grayscale
                plt.imshow(predict_image[0, :, :, ix - 1], cmap='gray')
                ix += 1
                
        return plt



    layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])

    output_layer = layer_dict[layer_name]
    model = Model(inputs=model.inputs, outputs=output_layer.output)
    model.summary()
    filter = model.get_weights()[1].shape
    class_id = model.predict(image)
    saved_img = draw_activations(filter, class_id)

    return saved_img




# Callbacks 
Keras has a callback fuction which you can call for each epoch this give you flexibility to add you custom fucntion after every training epoch has been finished, I have writen a custom keras callback function which after every 10th epoch will create and save the visualization results.
Here the filiter visualization fucntion is already available in https://keras.io/examples/conv_filter_visualization/. I have made some changes in it, which you can see in my github

In [4]:
class Visualization(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        model = self.model
        print(epoch)
        if epoch % 10 == 0 and epoch > 0:

            for i in range(len(model.layers)):
                if i == 0:
                    # Skipping the input layer
                    continue
                layer = model.layers[i]
                ''' check for convolutional layer, also restricting it to some of the layers
                    listed in above list this is to save time as the cnn layers had same depth I have decided not 
                    to visualise them you can change it and can save it for each and every layer'''
                if 'conv' not in layer.name or layer.name not in conv_layer_name:
                    continue

                # summarize output shape
                print(i, layer.name, layer.output.shape)
                save_dir = layer.name
                # This is the pre created keras fucntion to visualise filters.
                visualize_filter = visualization.visualize_layer(model, layer_name=layer.name)
                # This is custom visualization function defined earlier
                activation = Activation_visualistation(test_image, model,  layer.name)
                if not os.path.isdir(save_dir):
                    os.makedirs(save_dir)

                save_img(os.path.join(save_dir, "{0:}_{1:}.png".format(layer.name, epoch)), visualise_filter)
                # save_img(os.path.join(save_dir, "{0:}_activation_map_{1:}.png".format(layer.name, epoch)), activation)
                activation.savefig(os.path.join(save_dir, "{0:}_activation_map_{1:}.png".format(layer.name, epoch)))
                save_as_image(test_image, save_dir, 'test_image.png' )





In [5]:
model = create_model()


if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)
    history_cnn = tf.keras.callbacks.History()
    checkpoint_cnn = tf.keras.callbacks.ModelCheckpoint(filepath="saved_models/keras_cifar10_trained_model.h5", save_best_only=True,
                                                        monitor="val_loss", save_weights_only=False,
                                                        mode="min")
    vis = Visualization()
    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=100, use_multiprocessing=False,
                        callbacks=[checkpoint_cnn,
                                  ]

    )



Using real-time data augmentation.

Epoch 1/1000
98/98 [==============================] - 17s 168ms/step - loss: 2.2907 - accuracy: 0.1150 - val_loss: 2.2698 - val_accuracy: 0.2215
Epoch 2/1000
98/98 [==============================] - 13s 128ms/step - loss: 2.1274 - accuracy: 0.1991 - val_loss: 2.0859 - val_accuracy: 0.2388
Epoch 3/1000
98/98 [==============================] - 13s 133ms/step - loss: 2.0314 - accuracy: 0.2428 - val_loss: 1.9981 - val_accuracy: 0.2984
Epoch 4/1000
98/98 [==============================] - 13s 132ms/step - loss: 1.9557 - accuracy: 0.2803 - val_loss: 1.9057 - val_accuracy: 0.3362
Epoch 5/1000
98/98 [==============================] - 13s 133ms/step - loss: 1.8769 - accuracy: 0.3107 - val_loss: 1.8218 - val_accuracy: 0.3515
Epoch 6/1000
66/98 [===================>..........] - ETA: 4s - loss: 1.8292 - accuracy: 0.3251

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Saranjit\.conda\envs\carla\lib\site-packages\IPython\core\interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-64aa2f455274>", line 57, in <module>
    callbacks=[checkpoint_cnn,
  File "C:\Users\Saranjit\.conda\envs\carla\lib\site-packages\keras\legacy\interfaces.py", line 91, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\Saranjit\.conda\envs\carla\lib\site-packages\keras\engine\training.py", line 1732, in fit_generator
    initial_epoch=initial_epoch)
  File "C:\Users\Saranjit\.conda\envs\carla\lib\site-packages\keras\engine\training_generator.py", line 185, in fit_generator
    generator_output = next(output_generator)
  File "C:\Users\Saranjit\.conda\envs\carla\lib\site-packages\keras\utils\data_utils.py", line 610, in get
    inputs = future.get(timeout=30)
  File "C:\Users\Saranjit\.conda\envs\carla\lib\multiprocessing\pool.py", line 651, in ge

KeyboardInterrupt: 